# LEGENDE

'TotalGHGEmissions' : Quantité des emission de CO2 (par batiment et par an?) -> **variable cible**

'SiteEnergyUse(kBtu)' : energie total bat -> **variable cible**

'NaturalGas(kBtu)' : energie gas total bat

'LargestPropertyUseTypeGFA' : Consomation d'energie du local avec la surphase la plus importante

# IMPORTATION

In [1]:
import csv
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport #https://pandas-profiling.github.io/pandas-profiling/docs/master/index.html
#import qgrid

import pingouin as pg
import scipy
from pca import pca #pip install pca https://github.com/erdogant/pca

import ast #Tranfromer un dictioner

/home/chispitas/anaconda3/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.12, the latest is 0.5.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## STYLE Graph

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None) #Montrer toutes les colonnes

#using the style 
plt.style.use('seaborn-ticks')
plt.style.use('seaborn-notebook') 
plt.style.use('seaborn-poster') 
plt.rcParams["figure.figsize"] = [6, 5]

#Width
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['figure.titleweight'] = 'bold'
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'

#Legend
plt.rcParams['legend.loc'] = 'center left'
plt.rcParams['legend.fontsize'] = 'medium'
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.edgecolor'] = 'black'
plt.rcParams['legend.facecolor'] = 'w'
plt.rcParams['legend.markerscale'] = 3

#Scatter marker
plt.rcParams['scatter.marker'] = '.'
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'

In [3]:
#direc0 = '/media/chispitas/a534bfa2-2251-40fd-b2d7-9be320058604/Documento/Estudios/DataScientist/Projecto/P4-Anticipez les besoins en consommation electrique de batiments/Code/Data/'
direc0 = '/media/chispitas/DATOS/Documento/Estudios/DataScientist/Projecto/P4-Anticipez les besoins en consommation electrique de batiments/Code/Data/'

dir_15 = direc0 + '2015-building-energy-benchmarking.csv'
dir_16 = direc0 + '2016-building-energy-benchmarking.csv'

#dfDATA_Original_15 = pd.read_csv(dir_15, encoding='utf-8', nrows = 10) #, delimiter='\t'
dfDATA_Original_15 = pd.read_csv(dir_15, encoding='utf-8') #, delimiter='\t'
len(pd.unique(dfDATA_Original_15['OSEBuildingID']))
dfDATA_Original_15.sample(5)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Location,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),OtherFuelUse(kBtu),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes
2440,25563,2015,NonResidential,Non-Refrigerated Warehouse,NAUTICAL LANDING,4088804350,"{'latitude': '47.64257724', 'longitude': '-122...",7,LAKE UNION,1968,1,2.0,29498,0,29498,"Non-Refrigerated Warehouse, Office",Non-Refrigerated Warehouse,19749.0,Office,9148.0,NaN,NaN,NaN,NaN,119.6,119.6,340.1,340.1,3456208.0,3456208.0,0.0,869327.0,2966265.0,4901.0,490066.0,0.0,46.71,1.15,No,NaN,Compliant,NaN,NaN,50.0,NaN,7.0,18390
42,56,2015,NonResidential,Hotel,HILTON SEATTLE,1975700040,"{'latitude': '47.60950466', 'longitude': '-122...",7,DOWNTOWN,1969,1,24.0,332210,205970,126240,Hotel,Hotel,348630.0,NaN,NaN,NaN,NaN,NaN,68.0,44.9,44.9,121.0,121.0,15660628.0,15660628.0,0.0,3607318.0,12308680.0,33525.0,3352459.0,0.0,263.85,0.63,No,NaN,Compliant,NaN,NaN,14.0,NaN,24.0,18081
2460,25634,2015,Multifamily MR (5-9),Mid-Rise Multifamily,ACTON APTS,9221400410,"{'latitude': '47.67430413', 'longitude': '-122...",4,NORTHEAST,1992,1,5.0,30169,5668,24501,"Multifamily Housing, Parking, Retail Store",Multifamily Housing,18728.0,Retail Store,5773.0,Parking,5668.0,NaN,100.0,15.6,16.7,48.9,52.4,381370.0,408691.0,0.0,111773.0,381386.0,0.0,0.0,0.0,2.66,0.03,No,NaN,Compliant,Low Outlier,NaN,53.0,NaN,47.0,18792
2348,25245,2015,Multifamily LR (1-4),Low-Rise Multifamily,VICTORIA CONDOMINIUM,8898530000,"{'latitude': '47.62994973', 'longitude': '-122...",7,MAGNOLIA / QUEEN ANNE,1921,1,4.0,104064,0,104064,"Multifamily Housing, Other",Multifamily Housing,104064.0,Other,400.0,NaN,NaN,NaN,65.0,43.8,53.0,64.2,75.9,4571388.0,5538065.0,0.0,267496.0,912735.0,36587.0,3658690.0,0.0,200.68,1.89,No,NaN,Compliant,NaN,NaN,50.0,NaN,39.0,19575
1807,23420,2015,Multifamily MR (5-9),Mid-Rise Multifamily,1300 ALKI CONDOMINIUM,8600050000,"{'latitude': '47.59296293', 'longitude': '-122...",1,SOUTHWEST,1996,1,5.0,23821,0,23821,Multifamily Housing,Multifamily Housing,23821.0,NaN,NaN,NaN,NaN,NaN,NaN,37.1,41.9,69.5,74.5,884084.0,997547.0,0.0,102093.0,348355.0,5357.0,535743.0,0.0,30.88,1.23,No,NaN,Compliant,NaN,NaN,2.0,NaN,49.0,18795


In [4]:
dfDATA_Original_16 = pd.read_csv(dir_16, encoding='utf-8')
dfDATA_Original_16.sample(5)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
2062,24289,2016,Multifamily LR (1-4),Low-Rise Multifamily,Olympic View Apts,4517 Glenn Way SW,Seattle,WA,98116.0,3389900435,1,SOUTHWEST,47.561940,-122.389190,1969,1.0,4,26280,0,26280,Multifamily Housing,Multifamily Housing,24695.0,NaN,NaN,NaN,NaN,NaN,81.0,33.099998,35.599998,80.500000,87.800003,816510.875,8.793478e+05,0.0,1.584704e+05,540701.0,2758.100098,275810.0,False,NaN,Compliant,NaN,18.42,0.70
2071,24354,2016,NonResidential,Warehouse,1760 4th Avenue South,1760 4th Avenue South,Seattle,WA,98134.0,7666204565,2,GREATER DUWAMISH,47.586610,-122.328100,1989,1.0,1,23792,0,23792,"Non-Refrigerated Warehouse, Office",Non-Refrigerated Warehouse,22992.0,Office,800.0,NaN,NaN,NaN,NaN,119.900002,118.099998,376.399994,370.899994,2852102.750,2.810298e+06,0.0,8.359034e+05,2852102.0,0.000000,0.0,False,NaN,Compliant,NaN,19.88,0.84
3174,49714,2016,Multifamily MR (5-9),Mid-Rise Multifamily,Barclay Broadway,412 Broadway,Seattle,WA,98122.0,2197600185,3,EAST,47.605705,-122.320351,2012,1.0,6,94933,0,94933,"Multifamily Housing, Parking",Multifamily Housing,100419.0,Parking,31709.0,NaN,NaN,NaN,100.0,27.500000,29.100000,65.500000,68.900002,2757855.500,2.921981e+06,0.0,5.163123e+05,1761658.0,9961.975586,996198.0,False,NaN,Compliant,NaN,65.19,0.69
2786,26940,2016,Multifamily LR (1-4),Low-Rise Multifamily,Spruce Park Apartments,155 21st Avenue,Seattle,WA,98122.0,9826701275,3,CENTRAL,47.602890,-122.305280,1912,1.0,4,24980,0,24980,Multifamily Housing,Multifamily Housing,24980.0,NaN,NaN,NaN,NaN,NaN,44.0,53.900002,57.099998,126.800003,135.100006,1346978.750,1.426110e+06,0.0,2.459876e+05,839310.0,5076.689941,507669.0,False,NaN,Compliant,NaN,32.81,1.31
3228,49829,2016,NonResidential,Senior Care Community,Aegis on Madison,2200 E Madison,Seattle,WA,98112.0,2825049005,3,Central,47.618850,-122.303410,2013,1.0,6,79805,0,79805,"Parking, Senior Care Community",Senior Care Community,79805.0,Parking,7000.0,NaN,NaN,NaN,25.0,94.400002,96.400002,221.199997,223.300003,7531325.000,7.692164e+06,0.0,1.366841e+06,4663661.0,28676.628910,2867663.0,False,NaN,Compliant,NaN,184.81,2.32


# DATA PREPARATION

## COMBINING data

### Unify ID

In [5]:
dfDATA_Original_15 = dfDATA_Original_15.set_index('OSEBuildingID')
dfDATA_Original_16 = dfDATA_Original_16.set_index('OSEBuildingID')

### Unify variable

In [6]:
dfDATA_Original_16 = dfDATA_Original_16.rename(columns = {'ZipCode':'Zip Codes', 'Comments':'Comment'})
dfDATA_Original_16 = dfDATA_Original_16.rename(columns = {'TotalGHGEmissions' :'GHGEmissions(MetricTonsCO2e)'})

In [7]:
dfDATA_Original_15[['Zip Codes', 'Comment']].columns
dfDATA_Original_16[['Zip Codes', 'Comment']].columns 

Index(['Zip Codes', 'Comment'], dtype='object')

In [8]:
dfDATA_Original_16 = dfDATA_Original_16.drop(['PropertyGFATotal'], axis=1)
dfDATA_Original_15 = dfDATA_Original_15.drop(['PropertyGFATotal'], axis=1)
dfDATA_Original_15.columns

Index(['DataYear', 'BuildingType', 'PrimaryPropertyType', 'PropertyName',
       'TaxParcelIdentificationNumber', 'Location', 'CouncilDistrictCode',
       'Neighborhood', 'YearBuilt', 'NumberofBuildings', 'NumberofFloors',
       'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType',
       'ThirdLargestPropertyUseTypeGFA', 'YearsENERGYSTARCertified',
       'ENERGYSTARScore', 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
       'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)', 'Electricity(kWh)',
       'Electricity(kBtu)', 'NaturalGas(therms)', 'NaturalGas(kBtu)',
       'OtherFuelUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)',
       'GHGEmissionsIntensity(kgCO2e/ft2)', 'DefaultData', 'Comment',
       'ComplianceStatus', 'Outlie

### Reorganisation de la Var Location df2015 (LON, LAT,...)

In [9]:
#Exemple
gps = ast.literal_eval(dfDATA_Original_15.Location.loc[10])
address = ast.literal_eval(gps['human_address'])
print(dfDATA_Original_15.Location.loc[10])
print(gps['latitude'])
print(gps['human_address'])
print(address['city'])
print(gps)

{'latitude': '47.6141141', 'longitude': '-122.33274086', 'human_address': '{"address": "1619 9TH AVE", "city": "SEATTLE", "state": "WA", "zip": "98101"}'}
47.6141141
{"address": "1619 9TH AVE", "city": "SEATTLE", "state": "WA", "zip": "98101"}
SEATTLE
{'latitude': '47.6141141', 'longitude': '-122.33274086', 'human_address': '{"address": "1619 9TH AVE", "city": "SEATTLE", "state": "WA", "zip": "98101"}'}


In [10]:
for i in dfDATA_Original_15.index:
    gps = ast.literal_eval(dfDATA_Original_15.Location.loc[i])
    address = ast.literal_eval(gps['human_address'])
    #dfDATA_Original_15.loc[i,'Latitude'] = gps['latitude']
    dfDATA_Original_15.loc[i,'Latitude'] = pd.to_numeric(gps['latitude'], errors='coerce', downcast='unsigned').round(5)
    dfDATA_Original_15.loc[i,'Longitude'] = pd.to_numeric(gps['longitude'], errors='coerce', downcast='unsigned').round(5)
    dfDATA_Original_15.loc[i,'Address'] = address['address']
    dfDATA_Original_15.loc[i,'City'] = address['city']
    dfDATA_Original_15.loc[i,'State'] = address['state']
    dfDATA_Original_15.loc[i,'Zip Codes'] = pd.to_numeric(address['zip'], errors='coerce', downcast='unsigned').round(0) 

dfDATA_Original_15 = dfDATA_Original_15.drop(columns=['Location'])
dfDATA_Original_15.sample(5)

,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),OtherFuelUse(kBtu),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes,Latitude,Longitude,Address,City,State
OSEBuildingID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
25693,2015,Multifamily LR (1-4),Low-Rise Multifamily,KELSEY APARTMENTS,4092301481,4,NORTHEAST,1986,1,4.0,12959,34373,Multifamily Housing,Multifamily Housing,33822.0,NaN,NaN,NaN,NaN,NaN,93.0,30.9,34.5,97.1,108.5,1045640.0,1168488.0,0.0,306460.0,1045684.0,0.0,0.0,0.0,7.29,0.06,No,NaN,Compliant,NaN,NaN,60.0,NaN,38.0,98105,47.65862,-122.31909,4233 9TH AVE NE,SEATTLE,WA
28020,2015,NonResidential,K-12 School,OUR LADY OF FATIMA SCHOOL & GYM,6822100520,7,MAGNOLIA / QUEEN ANNE,1953,1,1.0,0,20811,K-12 School,K-12 School,20811.0,NaN,NaN,NaN,NaN,NaN,82.0,57.6,71.7,112.9,130.4,1198630.0,1492158.0,0.0,152984.0,522002.0,6766.0,676649.0,0.0,39.58,1.79,Yes,NaN,Compliant,NaN,NaN,33.0,NaN,27.0,98199,47.64831,-122.40000,3301 W DRAVUS ST,SEATTLE,WA
26061,2015,Multifamily LR (1-4),Low-Rise Multifamily,CASTELLAN,6848200745,3,EAST,1990,1,3.0,9438,27447,"Multifamily Housing, Parking",Multifamily Housing,27447.0,Parking,9438.0,NaN,NaN,NaN,83.0,27.8,31.5,87.4,98.8,763693.0,863868.0,0.0,223826.0,763725.0,0.0,0.0,0.0,5.32,0.06,Yes,NaN,Compliant,NaN,NaN,8.0,NaN,5.0,98102,47.62175,-122.32374,607 E HARRISON ST,SEATTLE,WA
20986,2015,NonResidential,Non-Refrigerated Warehouse,ORCAS BUILDING - KING COUNTY,1722800985,2,GREATER DUWAMISH,1968,1,1.0,0,27680,"Non-Refrigerated Warehouse, Office",Non-Refrigerated Warehouse,25000.0,Office,2680.0,NaN,NaN,NaN,NaN,61.9,73.3,113.4,128.8,1712796.0,2029603.0,0.0,187856.0,640990.0,10718.0,1071832.0,0.0,61.39,2.12,No,NaN,Compliant,NaN,NaN,22.0,NaN,37.0,98108,47.55048,-122.32379,707 S ORCAS ST,SEATTLE,WA
20140,2015,Multifamily LR (1-4),Low-Rise Multifamily,GALE PLACE APTS,1282301210,2,SOUTHEAST,1956,1,2.0,0,20400,"Multifamily Housing, Parking",Multifamily Housing,20448.0,Parking,0.0,NaN,NaN,NaN,42.0,46.3,51.9,145.5,162.8,947518.0,1060432.0,0.0,277702.0,947557.0,0.0,0.0,0.0,6.61,0.12,No,NaN,Compliant,NaN,NaN,38.0,NaN,42.0,98144,47.57285,-122.29208,3333 GALE PL S,SEATTLE,WA


### Joing data sets

In [11]:
#Update null elements with value in the same location in other.
dfDATA2 = dfDATA_Original_16.combine_first(dfDATA_Original_15)
dfDATA2.sample(5)

,2010 Census Tracts,Address,BuildingType,City,City Council Districts,Comment,ComplianceStatus,CouncilDistrictCode,DataYear,DefaultData,ENERGYSTARScore,Electricity(kBtu),Electricity(kWh),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity,GHGEmissionsIntensity(kgCO2e/ft2),LargestPropertyUseType,LargestPropertyUseTypeGFA,Latitude,ListOfAllPropertyUseTypes,Longitude,NaturalGas(kBtu),NaturalGas(therms),Neighborhood,NumberofBuildings,NumberofFloors,OtherFuelUse(kBtu),Outlier,PrimaryPropertyType,PropertyGFABuilding(s),PropertyGFAParking,PropertyName,SPD Beats,Seattle Police Department Micro Community Policing Plan Areas,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),State,SteamUse(kBtu),TaxParcelIdentificationNumber,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearBuilt,YearsENERGYSTARCertified,Zip Codes
OSEBuildingID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27225,NaN,2203 California Ave SW,Multifamily LR (1-4),Seattle,NaN,NaN,Compliant,1,2016,False,50.0,494951.0,145061.7969,21.74,0.92,0.78,Multifamily Housing,23671.0,47.58383,Multifamily Housing,-122.38681,344411.0,3444.109619,SOUTHWEST,1.0,3.0,0.0,NaN,Low-Rise Multifamily,23671,0,Admiral Apartments,49.0,41.0,NaN,NaN,35.500000,38.400002,8.393620e+05,9.089045e+05,80.900002,89.000000,WA,0.0,4432600005,NaN,NaN,1926,NaN,98116.0
24668,NaN,"531 BELLEVUE AVE E,",Multifamily LR (1-4),Seattle,NaN,NaN,Compliant,3,2016,False,81.0,431493.0,126463.5000,15.26,0.59,0.51,Multifamily Housing,25800.0,47.62414,Multifamily Housing,-122.32717,230603.0,2306.030029,EAST,1.0,3.0,0.0,NaN,Low-Rise Multifamily,25800,0,MARWOOD APTS (ID24668),5.0,8.0,NaN,NaN,25.700001,28.000000,6.620964e+05,7.228062e+05,61.900002,68.699997,WA,0.0,6848200100,NaN,NaN,1927,NaN,98102.0
24161,NaN,6701 21st Ave SW,Multifamily LR (1-4),Seattle,NaN,NaN,Compliant,1,2016,False,38.0,850616.0,249301.2031,5.93,0.23,0.09,Multifamily Housing,25951.0,47.54250,Multifamily Housing,-122.36012,0.0,0.000000,DELRIDGE,1.0,2.0,0.0,NaN,Low-Rise Multifamily,25951,0,Croft Place,13.0,45.0,NaN,NaN,32.799999,35.400002,8.506158e+05,9.176685e+05,102.900002,111.000000,WA,0.0,3438501950,NaN,NaN,2005,NaN,98106.0
20387,NaN,801 NW 42nd Street,NonResidential,Seattle,NaN,NaN,Compliant,6,2016,False,NaN,1553414.0,455279.5938,26.11,0.67,0.64,Non-Refrigerated Warehouse,16826.0,47.65742,"Non-Refrigerated Warehouse, Office",-122.36736,287740.0,2877.399902,BALLARD,1.0,2.0,0.0,NaN,Mixed Use Property,38784,0,Marine Business Center,2.0,4.0,Office,16826.0,54.700001,56.299999,1.841154e+06,1.894319e+06,153.899994,155.600006,WA,0.0,1325039001,NaN,NaN,2007,NaN,98107.0
25046,NaN,1311 S Massachusetts St,Multifamily HR (10+),Seattle,NaN,NaN,Compliant,2,2016,False,75.0,2154352.0,631404.3125,71.71,0.97,0.83,Multifamily Housing,79912.0,47.58820,Multifamily Housing,-122.31551,1067402.0,10674.020510,GREATER DUWAMISH,1.0,15.0,0.0,NaN,High-Rise Multifamily,73898,0,Beacon Tower,41.0,42.0,NaN,NaN,40.299999,42.799999,3.221754e+06,3.419862e+06,98.699997,105.500000,WA,0.0,7548800025,NaN,NaN,1971,NaN,98144.0


#### Verification update 2016

In [12]:
dfE15 = dfDATA_Original_15.rename(columns = {'SiteEnergyUse(kBtu)':'2015'})
dfE16 = dfDATA_Original_16.rename(columns = {'SiteEnergyUse(kBtu)':'2016'})

Verif = pd.concat([dfE15['2015'],dfE16['2016'],dfDATA2['SiteEnergyUse(kBtu)']], axis=1)
Verif = Verif.rename(columns = {'SiteEnergyUse(kBtu)':'Upgrade'})
Verif.sample(10).style.format('{0:,.0f}').background_gradient()

,2015,2016,Upgrade
OSEBuildingID,,,
24375,"1,618,570","1,436,510","1,436,510"
26748,"1,716,674","1,730,431","1,730,431"
239,"1,362,993","1,407,277","1,407,277"
50027,"1,725,773","1,066,277","1,066,277"
25529,"6,750,112","7,325,166","7,325,166"
760,"5,207,349","3,775,188","3,775,188"
314,"24,060,332","25,453,690","25,453,690"
21557,"3,886,929","3,485,866","3,485,866"
325,"9,898,724",nan,"9,898,724"


## DATA cleaning

In [13]:
# Organisation des variables (NUM/STR,DISCRETES/CONTINUES)
def lVAR_classification(dfSET):
 
    lVAR_cible = [ ] #Variable CIBLES 

    lVAR_str = dfSET.select_dtypes(exclude=[np.number, np.int64, 'datetime', 'timedelta']).columns.difference(
        lVAR_cible+[])  #Les variables str DISCRETES ET CONTINUES
    lVAR_num = dfSET.select_dtypes(include=[np.number, np.int64], exclude=['datetime', 'timedelta']).columns.difference(
        lVAR_cible+[])  #Les variables floats DISCRETES ET CONTINUES

    lVAR_str_cat = dfSET[lVAR_str].nunique().sort_values()[(dfSET.nunique()<500)].index.difference([ ]) #Les variables str DISCRETES
    lVAR_str_cont  = lVAR_str.difference(lVAR_str_cat) #Les variables floats CONTINUES

    lVAR_num_cat = dfSET[lVAR_num].nunique().sort_values()[(dfSET.nunique()<500)].index.difference(
        []) #Les variables floats DISCRETES
    lVAR_num_cont = lVAR_num.difference(lVAR_num_cat) #Les variables floats CONTINUES

    lVAR_cat = np.concatenate((lVAR_str_cat, lVAR_num_cat), axis=None)
    lVAR_cont = np.concatenate((lVAR_str_cont, lVAR_num_cont), axis=None)
    
    print('lVAR_cible',lVAR_cible)
    print('.......')
    print('lVAR_str_cat ',lVAR_str_cat)
    print('lVAR_str_cont ',lVAR_str_cont)
    print('.......')
    print('lVAR_num_cat ',lVAR_num_cat)
    print('lVAR_num_cont ',lVAR_num_cont)
    print('.......')
    print('lVAR_cat ',lVAR_cat)
    print('lVAR_cont ',lVAR_cont)
    
    return lVAR_cible, lVAR_str, lVAR_num, lVAR_cat, lVAR_cont, lVAR_str_cat, lVAR_str_cont, lVAR_num_cat, lVAR_num_cont

In [15]:
lVAR_cible, lVAR_str, lVAR_num, lVAR_cat, lVAR_cont, lVAR_str_cat, lVAR_str_cont, lVAR_num_cat, lVAR_num_cont = lVAR_classification(
    dfDATA2[dfDATA2.columns.difference([''])])

lVAR_cible []
.......
lVAR_str_cat  Index(['State', 'City', 'ComplianceStatus', 'DefaultData', 'Outlier',
       'BuildingType', 'Comment', 'Neighborhood', 'PrimaryPropertyType',
       'ThirdLargestPropertyUseType', 'SecondLargestPropertyUseType',
       'LargestPropertyUseType', 'YearsENERGYSTARCertified',
       'ListOfAllPropertyUseTypes'],
      dtype='object')
lVAR_str_cont  Index(['Address', 'PropertyName', 'TaxParcelIdentificationNumber'], dtype='object')
.......
lVAR_num_cat  Index(['City Council Districts', 'DataYear', 'CouncilDistrictCode',
       '2010 Census Tracts', 'NumberofBuildings', 'OtherFuelUse(kBtu)',
       'NumberofFloors', 'SPD Beats', 'Zip Codes',
       'Seattle Police Department Micro Community Policing Plan Areas',
       'ENERGYSTARScore', 'YearBuilt', 'SteamUse(kBtu)',
       'GHGEmissionsIntensity(kgCO2e/ft2)'],
      dtype='object')
lVAR_num_cont  Index(['Electricity(kBtu)', 'Electricity(kWh)', 'GHGEmissions(MetricTonsCO2e)',
       'GHGEmissionsIntensit

### Spaces, typos, ...

#### srt var identification

In [16]:
for c in lVAR_str :
    print(c)
    dfDATA2[c] = dfDATA2[c].str.lower()
    dfDATA2[c] = dfDATA2[c].str.strip(' ')
dfDATA2.sample(5)

Address
BuildingType
City
Comment
ComplianceStatus
DefaultData
LargestPropertyUseType
ListOfAllPropertyUseTypes
Neighborhood
Outlier
PrimaryPropertyType
PropertyName
SecondLargestPropertyUseType
State
TaxParcelIdentificationNumber
ThirdLargestPropertyUseType
YearsENERGYSTARCertified


,2010 Census Tracts,Address,BuildingType,City,City Council Districts,Comment,ComplianceStatus,CouncilDistrictCode,DataYear,DefaultData,ENERGYSTARScore,Electricity(kBtu),Electricity(kWh),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity,GHGEmissionsIntensity(kgCO2e/ft2),LargestPropertyUseType,LargestPropertyUseTypeGFA,Latitude,ListOfAllPropertyUseTypes,Longitude,NaturalGas(kBtu),NaturalGas(therms),Neighborhood,NumberofBuildings,NumberofFloors,OtherFuelUse(kBtu),Outlier,PrimaryPropertyType,PropertyGFABuilding(s),PropertyGFAParking,PropertyName,SPD Beats,Seattle Police Department Micro Community Policing Plan Areas,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),State,SteamUse(kBtu),TaxParcelIdentificationNumber,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearBuilt,YearsENERGYSTARCertified,Zip Codes
OSEBuildingID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27059,NaN,700 broadway east,multifamily lr (1-4),seattle,NaN,NaN,compliant,3,2016,NaN,55.0,2107108.0,617558.1250,43.28,0.49,0.38,multifamily housing,58849.0,47.625651,"bank branch, multifamily housing, parking, res...",-122.32065,538373.0,5383.729492,east,1.0,4.0,0.0,NaN,low-rise multifamily,56867,31559,700 broadway,4.0,44.0,parking,29868.0,38.200001,39.500000,2645481.25,2740614.000,103.599998,107.300003,wa,0.0,9831200640,strip mall,4312.0,2003,NaN,98102.0
24541,NaN,300 10th ave,multifamily lr (1-4),seattle,NaN,NaN,compliant,3,2016,NaN,89.0,1604063.0,470123.9063,11.18,0.23,0.09,multifamily housing,66257.0,47.604920,multifamily housing,-122.31898,0.0,0.000000,east,1.0,4.0,0.0,NaN,low-rise multifamily,48972,0,the pointe at first hill homeowners association,16.0,19.0,NaN,NaN,24.200001,25.100000,1604062.75,1661746.625,76.000000,78.800003,wa,0.0,6837810000,NaN,NaN,1990,NaN,98122.0
22118,NaN,"4515 mlk jr. way so., ste. 200",multifamily lr (1-4),seattle,NaN,NaN,compliant,2,2016,NaN,91.0,1200508.0,351848.6875,44.61,0.56,0.54,multifamily housing,77823.0,47.562340,multifamily housing,-122.29469,682338.0,6823.379395,greater duwamish,1.0,4.0,0.0,NaN,low-rise multifamily,80155,0,providence gamelin house,42.0,10.0,NaN,NaN,24.200001,25.200001,1882845.75,1957926.750,57.599998,59.900002,wa,0.0,2692000000,NaN,NaN,2004,NaN,98108.0
23028,NaN,2101-2111 first avenue & 92 lenora street,multifamily lr (1-4),seattle,NaN,NaN,compliant,7,2016,NaN,NaN,1169932.0,342887.5938,8.16,0.19,0.07,multifamily housing,21000.0,47.611810,"multifamily housing, office, retail store",-122.34417,0.0,0.000000,downtown,1.0,3.0,0.0,NaN,mixed use property,44104,0,m0010 & m0040 - the guiry schillestad,9.0,5.0,office,11100.0,27.100000,29.400000,1169932.75,1268988.250,85.000000,92.199997,wa,0.0,2953700000,retail store,11100.0,1901,NaN,98121.0
26657,NaN,1604 ne 50th st,nonresidential,seattle,NaN,NaN,compliant,4,2016,NaN,NaN,432910.0,126878.6016,37.43,0.91,1.02,worship facility,32098.0,47.665170,"office, worship facility",-122.31037,647911.0,6479.110352,northeast,1.0,3.0,0.0,NaN,worship facility,41013,0,university lutheran church,47.0,60.0,office,2310.0,31.400000,34.599998,1080821.00,1191549.250,59.299999,62.700001,wa,0.0,8823901075,NaN,NaN,1927,NaN,98105.0


#### Last year of 'YearsENERGYSTARCertified'

In [17]:
dfDATA2.loc[368,'YearsENERGYSTARCertified'][:4]
pd.notnull(dfDATA2.loc[2,'YearsENERGYSTARCertified'])

False

In [18]:
dfDATA2.loc[482,'YearsENERGYSTARCertified']
for i in dfDATA2.index:
    if pd.notnull(dfDATA2.loc[i,'YearsENERGYSTARCertified']): 
        dfDATA2.loc[i,'YearsENERGYSTARCertified'] = dfDATA2.loc[i,'YearsENERGYSTARCertified'][:4]

dfDATA2['YearsENERGYSTARCertified'] = pd.to_numeric(dfDATA2['YearsENERGYSTARCertified'], errors='coerce', downcast='unsigned').round(0).astype('float32') 
dfDATA2['YearsENERGYSTARCertified'].sample(20)


OSEBuildingID
25354       NaN
27602       NaN
24494       NaN
23075       NaN
331      2010.0
27456       NaN
410         NaN
25918       NaN
570         NaN
25529       NaN
25455       NaN
25240       NaN
19911       NaN
20658       NaN
27406       NaN
25245       NaN
26593       NaN
27570       NaN
569      2010.0
49800       NaN
Name: YearsENERGYSTARCertified, dtype: float32

profile = ProfileReport(dfDATA2, minimal=True)
profile.to_file("dfDATA2.html")

### Missing values

In [20]:
dfDESCRIPTION = dfDATA2.describe(include='all')
dfTAUX = dfDESCRIPTION.T
LEN = len(dfDATA2)
dfTAUX['%full'] = (dfTAUX['count']/LEN)*100
dfTAUX['%unique'] = (1-dfTAUX['unique']/dfTAUX['count'])*100
dfTAUX.drop(['top'], axis=1,inplace = True)

In [21]:
dfTAUX.sort_values('%full', ascending = False ,inplace = True)
dfTAUX[['count', 'unique', 'freq', 'mean', 'std', 'min', '25%', '50%','75%', 'max', '%full', '%unique']].style.format('{0:,.1f}').background_gradient(subset=['count', 'unique', 'freq', 'mean', 'std', 'min', '25%', '50%','75%', 'max', '%full', '%unique'])

,count,unique,freq,mean,std,min,25%,50%,75%,max,%full,%unique
NumberofFloors,"3,432.0",nan,nan,4.7,5.5,0.0,2.0,4.0,5.0,99.0,100.0,nan
State,"3,432.0",1.0,"3,432.0",nan,nan,nan,nan,nan,nan,nan,100.0,100.0
PrimaryPropertyType,"3,432.0",26.0,"1,009.0",nan,nan,nan,nan,nan,nan,nan,100.0,99.2
PropertyName,"3,432.0","3,415.0",3.0,nan,nan,nan,nan,nan,nan,nan,100.0,0.5
Address,"3,432.0","3,408.0",4.0,nan,nan,nan,nan,nan,nan,nan,100.0,0.7
Neighborhood,"3,432.0",14.0,586.0,nan,nan,nan,nan,nan,nan,nan,100.0,99.6
Longitude,"3,432.0",nan,nan,-122.3,0.0,-122.4,-122.4,-122.3,-122.3,-122.2,100.0,nan
Latitude,"3,432.0",nan,nan,47.6,0.0,47.5,47.6,47.6,47.7,47.7,100.0,nan
PropertyGFAParking,"3,432.0",nan,nan,"8,166.8","33,029.2",0.0,0.0,0.0,0.0,"512,608.0",100.0,nan
PropertyGFABuilding(s),"3,432.0",nan,nan,"86,416.7","206,449.8","3,636.0","27,639.8","43,137.0","84,034.5","9,320,156.0",100.0,nan


#### Supression des variables avec un taux < taux de 'YearsENERGYSTARCertified'

In [26]:
lVAR_sup = dfTAUX[dfTAUX['%full'] < 50].index
print(lVAR_sup)
dfDATA2[lVAR_sup].sample(10) #DF des variables suprimes

Index(['2010 Census Tracts', 'City Council Districts', 'Comment',
       'DefaultData', 'Outlier', 'SecondLargestPropertyUseType',
       'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType',
       'ThirdLargestPropertyUseTypeGFA', 'YearsENERGYSTARCertified'],
      dtype='object')


,2010 Census Tracts,City Council Districts,Comment,DefaultData,Outlier,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified
OSEBuildingID,,,,,,,,,,
769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50061,NaN,NaN,NaN,NaN,NaN,retail store,6340.0,NaN,NaN,NaN
21688,NaN,NaN,NaN,no,NaN,medical office,8315.0,NaN,NaN,NaN
21391,NaN,NaN,NaN,NaN,NaN,other - services,7016.0,restaurant,3609.0,NaN
43867,NaN,NaN,NaN,NaN,high outlier,restaurant,8700.0,parking,8500.0,NaN
20402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49994,NaN,NaN,NaN,NaN,NaN,parking,75138.0,NaN,NaN,NaN
25364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27298,117.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
lVAR_sup = dfTAUX[dfTAUX['%full'] < 50].index
print(lVAR_sup)
dfDATA2[lVAR_sup].sample(10) #DF des variables suprimes

In [28]:
#Je garde : 'YearsENERGYSTARCertified', 'Longitude', 'Latitude'
lVAR_sup = lVAR_sup.difference(['YearsENERGYSTARCertified'])
dfDATA2 = dfDATA2.drop(columns=lVAR_sup)
dfTAUX = dfTAUX.drop(index=lVAR_sup)

In [ ]:
dfDATA2.columns

#### Supression des variables avec unique = 1

In [30]:
lVAR_sup = dfTAUX[dfTAUX['unique'] == 1 ].index
dfDATA2 = dfDATA2.drop(columns=lVAR_sup)
dfTAUX = dfTAUX.drop(lVAR_sup)

print(lVAR_sup)

Index(['State', 'City'], dtype='object')


In [ ]:
dfDATA2 = dfDATA2[dfDATA2['ComplianceStatus']=='compliant']
dfDATA2 = dfDATA2.drop('ComplianceStatus', axis=1)

dfDATA2 = dfDATA2.drop('DataYear', axis=1)

dfDATA2.columns

#### Supression des doublons

In [ ]:
print (dfDATA2.shape)
dfDATA2 = dfDATA2.drop_duplicates()
print (dfDATA2.shape)

#### Optimisation de la memoire (dtype)['YearsENERGYSTARCertified', 'YearBuilt']

dfDATA2 = dfDATA2.convert_dtypes()
dfDATA2.dtypes

In [ ]:
lVAR_float = dfDATA2.select_dtypes(include=[np.floating]).columns #Les variables floats

dfDATA2[lVAR_float] = dfDATA2[lVAR_float].round(5).astype('float32')
dfDATA2[lVAR_float]

#in8 = 127
#in16 = 30000
#in32 = 2*10^9
#in64 = 2*10^19

#dfTAUX[dfTAUX['max']<in8].index

#lVAR_int8 = ['NumberofBuildings', 'NumberofFloors']
lVAR_float = dfDATA2.select_dtypes(include=[np.floating]).columns #Les variables floats
lVAR_float64 = dfTAUX[(dfTAUX['max']>10**10) | (dfTAUX['min']<-10**10)].index
lVAR_float32 = dfTAUX[(dfTAUX['max']>10**5) | (dfTAUX['min']<-10**5)].index.difference(lVAR_float64)
lVAR_float16 = lVAR_float.difference(lVAR_float32)

#dfDATA2[lVAR_float64] = dfDATA2[lVAR_float64].round(5).astype('float64')
#dfDATA2[lVAR_float32] = dfDATA2[lVAR_float32].round(5).astype('float32')
#dfDATA2[lVAR_float16] = dfDATA2[lVAR_float16].round(5).astype('float16')
#dfDATA2[lVAR_int8] = dfDATA2[lVAR_int8].round(0).astype('int8')
dfDATA2[lVAR_float] = dfDATA2[lVAR_float].round(5).astype('float32')
dfDATA2.dtypes

In [ ]:
dfDESCRIPTION = dfDATA2.describe(include='all')
dfTAUX = dfDESCRIPTION.T
dfTAUX.sort_values('max', ascending = False ,inplace = True)
dfTAUX[['count', 'unique', 'freq', 'mean', 'std', 'min', '25%', '50%','75%', 'max']].style.format('{0:,.0f}').background_gradient(subset=['count', 'unique', 'freq', 'mean', 'std', 'min', '25%', '50%','75%', 'max'])

#### Filtrage des variables

In [ ]:
#Variables preselectiones:
lVAR_sup = ['Electricity(kWh)', 'NaturalGas(therms)','Seattle Police Department Micro Community Policing Plan Areas', 
            'SiteEUIWN(kBtu/sf)','SourceEUIWN(kBtu/sf)','SiteEnergyUseWN(kBtu)',
            'SiteEUI(kBtu/sf)','SourceEUI(kBtu/sf)',
            'SecondLargestPropertyUseType', 'LargestPropertyUseType', 'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA','LargestPropertyUseTypeGFA',
            'GHGEmissionsIntensity','GHGEmissionsIntensity(kgCO2e/ft2)',
            'Comment', 'Comments', 'DataYear', 'PropertyName', 'Address', '2010 Census Tracts', 'City Council Districts', 'ListOfAllPropertyUseTypes', 'TaxParcelIdentificationNumber']
lVAR_columns = dfDATA2.columns
lVAR_sup = lVAR_columns.intersection(lVAR_sup)
lVAR_sup

In [ ]:
dfDATA2 = dfDATA2.drop(lVAR_sup, axis=1)
dfDATA2

### Physically impossible values

#### Variables > 0

In [ ]:
lVAR_str = dfDATA2.select_dtypes(include=[np.object]).columns #Les variables str
lVAR_float = dfDATA2.select_dtypes(include=[np.floating]).columns #Les variables floats
lVAR_float

In [ ]:
for c in lVAR_float.difference(['Longitude', 'YearsENERGYSTARCertified']) :
    dfDATA2 = dfDATA2[dfDATA2[c]>=0]
dfDATA2

#### Chaque type d'energie < à l'energie total

In [ ]:
dfDATA2[dfDATA2['Electricity(kBtu)'] > dfDATA2['SiteEnergyUse(kBtu)']]

In [ ]:
dfDATA2.shape

In [ ]:
lVAR_enerie = ['Electricity(kBtu)', 'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)', 'SteamUse(kBtu)']
for c in lVAR_enerie:
    dfDATA2 = dfDATA2[dfDATA2[c] < dfDATA2['SiteEnergyUse(kBtu)']]
dfDATA2.shape

#### La sum des energies < à l'energie total

In [ ]:
dfDATA2['TotalEnergy(kBtu)'] = dfDATA2[lVAR_enerie].sum(axis=1, skipna=True, numeric_only=True)
dfDATA2[['Electricity(kBtu)', 'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)', 'SteamUse(kBtu)', 'TotalEnergy(kBtu)','SiteEnergyUse(kBtu)']]

In [ ]:
dfDATA2 = dfDATA2[dfDATA2['SiteEnergyUse(kBtu)'] <= dfDATA2['TotalEnergy(kBtu)']*1.01]
dfDATA2.shape

#### 'NumberofBuildings', 'NumberofFloors' +1

In [ ]:
dfDATA2[dfDATA2['NumberofBuildings']== 0].sample(5)

In [ ]:
dfDATA2[dfDATA2['NumberofFloors']== 0].sample(5)

In [ ]:
dfDATA2[(dfDATA2['NumberofBuildings']== 0) & (dfDATA2['NumberofFloors']== 0)]


In [ ]:
dfDATA2['NumberofBuildings'][dfDATA2['NumberofBuildings']== 0] = 1
dfDATA2['NumberofFloors'] = dfDATA2['NumberofFloors']+1

#### 'YearsENERGYSTARCertified', 'YearBuilt' >0

In [ ]:
dfDATA2[dfDATA2['YearBuilt'] < 1900]

In [ ]:
dfDATA2[dfDATA2['YearsENERGYSTARCertified'] < 1900]

In [ ]:
dfDATA2[dfDATA2['Zip Codes'] < 80000]

In [ ]:
dfDATA2 = dfDATA2[dfDATA2['YearBuilt'] > 1900]
dfDATA2 = dfDATA2[(dfDATA2['YearsENERGYSTARCertified'] > 1900) | pd.isnull(dfDATA2['YearsENERGYSTARCertified'])]
dfDATA2 = dfDATA2[dfDATA2['Zip Codes'] > 80000]

In [ ]:
dfDATA2.shape

### Outlier

In [ ]:
plt.style.use('seaborn-notebook') 

lVARS = lVAR_float.difference(['Longitude', 'Latitude'])
lVARS = dfDATA2.loc[:,lVARS].describe(include='all').T.sort_values('max', ascending = False).index #Les variables organiser par le plus grandes aux plus petites

nc = 4
nf = 9

fig = plt.figure(figsize=(7*nc, 7*nf))
plt.subplots_adjust(wspace=0.2, hspace=0.2) #interspace entre graphiques

cc = -1
for c in lVARS:
    cc = cc + 1
    
    ax = fig.add_subplot(nf, nc, (cc+1))
    ax = sns.boxplot(y=dfDATA2[c], showfliers=True, showmeans = True, ax = ax)
    ax.set_title(c)
    ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '32-Outliers.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')


In [ ]:
ax = sns.histplot(data=dfDATA2['NumberofBuildings'], common_norm=True, bins=30, kde=True)
ax.set_title('NumberofBuildings')
ax.set_ylabel('')
ax.set_xlabel('')
#ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '33-1D-Distribution-NumberofBuildings.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

In [ ]:
dfDATA2 = dfDATA2[dfDATA2['NumberofBuildings'] == 1]
dfDATA2 = dfDATA2.drop('NumberofBuildings', axis=1)
dfDATA2

### Errorrs against codebook

## DATA Transformation

In [ ]:
# Organisation des variables (FLOAT/STR,DISCRETES/CONTINUES)
lVAR_cible = ['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)', 'SiteEnergyUse(kBtu)/Floors', 'GHGEmissions(MetricTonsCO2e)/Floors'] #Variable CIBLES 

lVAR_str = dfDATA2.select_dtypes(exclude=[np.floating]).columns.difference(lVAR_cible)  #Les variables str DISCRETES ET CONTINUES
lVAR_float = dfDATA2.select_dtypes(include=[np.floating]).columns.difference(lVAR_cible)  #Les variables floats DISCRETES ET CONTINUES

lVAR_discret_str = dfDATA2[lVAR_str].nunique().sort_values()[(dfDATA2.nunique()<1000)].index.difference(['Comment']) #Les variables str DISCRETES
lVAR_continue_str  = lVAR_str.difference(lVAR_discret_str) #Les variables floats CONTINUES

lVAR_float_discret = dfDATA2[lVAR_float].nunique().sort_values()[(dfDATA2.nunique()<1000)].index.difference(['OtherFuelUse(kBtu)', 'SPD Beats', 'SteamUse(kBtu)', 'GHGEmissionsIntensity',
       'GHGEmissionsIntensity(kgCO2e/ft2)']) #Les variables floats DISCRETES
lVAR_float_continue = lVAR_float.difference(lVAR_float_discret) #Les variables floats CONTINUES

lVAR_discret = np.concatenate((lVAR_discret_str, lVAR_float_discret), axis=None)
lVAR_continue = np.concatenate((lVAR_continue_str, lVAR_float_continue), axis=None)

print('lVAR_discret_str ',lVAR_discret_str)
print('lVAR_continue_str ',lVAR_continue_str)

print('lVAR_float_discret ',lVAR_float_discret)
print('lVAR_float_continue ',lVAR_float_continue)

print('lVAR_discret ',lVAR_discret)
print('lVAR_continue ',lVAR_continue)

### Aggregating data

#### 'YearBuilt_10y' et 'YearsENERGYSTARCertified_10y'

In [ ]:
dfDATA2['YearBuilt_10y'] = np.nan
dfDATA2['YearsENERGYSTARCertified_10y'] = np.nan
#['YearsENERGYSTARCertified', 'YearBuilt']:
for iy in np.arange(1900,2030,10):
    cond1 = (dfDATA2['YearBuilt']>=iy) & (dfDATA2['YearBuilt']<=iy+9)
    cond2 = (dfDATA2['YearsENERGYSTARCertified']>=iy) & (dfDATA2['YearsENERGYSTARCertified']<=iy+9)
    if [cond1]: 
        dfDATA2['YearBuilt_10y'][cond1] = iy
    if [cond2]: 
        dfDATA2['YearsENERGYSTARCertified_10y'][cond2] = iy

#dfDATA2[['YearBuilt_10y', 'YearsENERGYSTARCertified_10y']] = dfDATA2[['YearBuilt_10y', 'YearsENERGYSTARCertified_10y']].round(0).astype('float16')
dfDATA2

#### 'SiteEnergyUse(kBtu)/m2' et 'GHGEmissions(MetricTonsCO2e)/m2'

dfDATA2['SiteEnergyUse(kBtu)/m2'] = dfDATA2['SiteEnergyUse(kBtu)']/dfDATA2['PropertyGFABuilding(s)']
dfDATA2['GHGEmissions(MetricTonsCO2e)/m2'] = dfDATA2['GHGEmissions(MetricTonsCO2e)']/dfDATA2['PropertyGFABuilding(s)']

### Extrapolating data

In [ ]:
dfDATA2.columns

### Derived measures

### Creating dummies

In [ ]:
dfDATA2.columns

### Reducing number of variables https://stackoverflow.com/questions/37815987/plot-a-correlation-circle-in-python 

#### Correlation ANOVA

In [ ]:
DATA_corr = dfDATA2.copy(deep=True) #.sample(5000)
# Organisation des variables (FLOAT/STR,DISCRETES/CONTINUES)
lVAR_cible = ['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)', 'SiteEnergyUse(kBtu)/Floors', 'GHGEmissions(MetricTonsCO2e)/Floors'] #Variable CIBLES 
lVAR_cible = ['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)'] #Variable CIBLES 

lVAR_str = DATA_corr.select_dtypes(exclude=[np.floating]).columns.difference(lVAR_cible)  #Les variables str DISCRETES ET CONTINUES
lVAR_float = DATA_corr.select_dtypes(include=[np.floating]).columns.difference(lVAR_cible)  #Les variables floats DISCRETES ET CONTINUES

lVAR_discret_str = DATA_corr[lVAR_str].nunique().sort_values()[(DATA_corr.nunique()<1000)].index.difference(['Comment']) #Les variables str DISCRETES
lVAR_continue_str  = lVAR_str.difference(lVAR_discret_str) #Les variables floats CONTINUES

lVAR_float_discret = DATA_corr[lVAR_float].nunique().sort_values()[(DATA_corr.nunique()<1000)].index.difference(['OtherFuelUse(kBtu)', 'SPD Beats', 'SteamUse(kBtu)', 'GHGEmissionsIntensity',
       'GHGEmissionsIntensity(kgCO2e/ft2)']) #Les variables floats DISCRETES
lVAR_float_continue = lVAR_float.difference(lVAR_float_discret) #Les variables floats CONTINUES

lVAR_discret = np.concatenate((lVAR_discret_str, lVAR_float_discret), axis=None)
lVAR_continue = np.concatenate((lVAR_continue_str, lVAR_float_continue), axis=None)

print('lVAR_discret_str ',lVAR_discret_str)
print('lVAR_continue_str ',lVAR_continue_str)

print('lVAR_float_discret ',lVAR_float_discret)
print('lVAR_float_continue ',lVAR_float_continue)

print('lVAR_discret ',lVAR_discret)
print('lVAR_continue ',lVAR_continue)

##### Test de normalité

###### Distribution

In [ ]:
plt.style.use('seaborn-notebook') 

lVARS = lVAR_float.difference(['Longitude', 'Latitude'])
lVARS = DATA_corr.loc[:,lVARS].describe(include='all').T.sort_values('max', ascending = False).index #Les variables organiser par le plus grandes aux plus petites

nc = 6
nf = 9

fig = plt.figure(figsize=(7*nc/2, 7*nf/2))
plt.subplots_adjust(wspace=0.4, hspace=0.4) #interspace entre graphiques

cc = -1
for c in lVARS:
    cc = cc + 1
    
    ax = fig.add_subplot(nf, nc, (cc+1))
    ax = sns.histplot(data=DATA_corr[c],ax = ax, common_norm=True, bins=30, kde=True)
    ax.set_title(c)
    ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '34-1D-Distribution-TOUT.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

#####  Shapiro–Wilk's test of normality
L’hypothèse nulle : La population est normalement distribuée.

Si pval < α choisi, alors l’hypothèse nulle est rejetée et il est prouvé que les données testées ne sont pas normalement distribuées.

Dans notre cas, **la probabilité que les variables soint normalement distribuées est de 0.00%**. pval < α (5%).
 


        'W': Test statistic.

        'pval': p-value.

        'normal': True if data is normally distributed.

In [ ]:
pg.normality(DATA_corr, alpha=0.05).style.format('{0:,.2f}', subset=['pval', 'W'])

**0.00% de probabilité que les variables soint normalement distribuées**

##### Visualisation de la normalité

In [ ]:
mean, std = 0, 0.8

plt.style.use('seaborn-notebook') 

lVARS = lVAR_float.difference(['Longitude', 'Latitude'])
#lVARS = DATA_corr.loc[:,lVARS].describe(include='all').T.sort_values('max', ascending = False).index #Les variables organiser par le plus grandes aux plus petites

nc = 5
nf = 20

fig = plt.figure(figsize=(7*nc/2, 7*nf/2))
plt.subplots_adjust(wspace=0.4, hspace=0.4) #interspace entre graphiques

cc = -1
for c in lVARS:
    cc = cc + 1
    
    ax = fig.add_subplot(nf, nc, (cc+1))
    ax = pg.qqplot(DATA_corr[c], dist='norm', sparams=(mean, std), ax=ax)
    ax.set_title(c)
    ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '42-Distribution-NormaliteQQ.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

##### Box-Cox

In [ ]:
mean, std = 0, 0.8

plt.style.use('seaborn-notebook') 

#lVARS = lVAR_float_continue.difference(['Longitude', 'Latitude', 'YearsENERGYSTARCertified', 'SPD Beats', 'NumberofFloors', 'ENERGYSTARScore', 'YearBuilt'])
lVARS = lVAR_float_continue.difference(['Longitude', 'Latitude', 'YearsENERGYSTARCertified'])
lVARS = DATA_corr.loc[:,lVARS].describe(include='all').T.sort_values('max', ascending = False).index #Les variables organiser par le plus grandes aux plus petites

nc = 5
nf = 20

fig = plt.figure(figsize=(7*nc/2, 7*nf/2))
plt.subplots_adjust(wspace=0.4, hspace=0.4) #interspace entre graphiques

cc = -1
for c in lVARS:
    cc = cc + 1

    print(c)
    DATA_boxcox = scipy.stats.boxcox(DATA_corr[c][(DATA_corr[c] > 0)].values.tolist())
    ax = fig.add_subplot(nf, nc, (cc+1))
    ax = pg.qqplot(DATA_boxcox[0], dist='norm', sparams=(mean, std), ax=ax)
    
    DATA_boxcox, lamda = scipy.stats.boxcox(DATA_corr[c][(DATA_corr[c] > 0)].values.tolist())
    ax.set_title(c + ' l=' +str(lamda.round(2)))
    ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '42-Distribution-NormaliteQQ-KB.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

##### Kruskal-Wallis test
L’hypothèse nulle : La médiane de population de tous les groupes est
égale (version non paramétrique de l’ANOVA).

Dans notre cas, la probabilité que les variables soint independants est
de 0.00% -> (la probabilité que les variables soint dependants est de
100%) -> Correlation.


In [ ]:
DATA_corr.columns

.# La fonction n'accepte pas les - ni ()
DATA_corr.columns = ['BuildingType', 'CouncilDistrictCode', 'ENERGYSTARScore',
       'Electricity_kBtu', 'GHGEmissions_MetricTonsCO2e', 'Latitude',
       'Longitude', 'NaturalGas_kBtu', 'Neighborhood', 'NumberofFloors',
       'OtherFuelUse_kBtu', 'PrimaryPropertyType', 'PropertyGFABuilding_s', 'PropertyGFAParking',
       'SPD Beats', 'SiteEnergyUse_kBtu', 'SteamUse_kBtu', 'YearBuilt',
       'YearsENERGYSTARCertified', 'Zip Codes', 'TotalEnergy_kBtu',
       'YearBuilt_10y', 'YearsENERGYSTARCertified_10y',
       'SiteEnergyUse_kBtu_m2', 'GHGEmissions_MetricTonsCO2e_m2']
DATA_corr

In [ ]:
# La fonction n'accepte pas les - ni ()
DATA_corr.columns = ['BuildingType', 'CouncilDistrictCode', 'ENERGYSTARScore',
       'Electricity_kBtu', 'GHGEmissions_MetricTonsCO2e', 'Latitude',
       'Longitude', 'NaturalGas_kBtu', 'Neighborhood', 'NumberofFloors',
       'OtherFuelUse_kBtu', 'PrimaryPropertyType', 'PropertyGFABuilding_s', 'PropertyGFAParking',
       'SPD Beats', 'SiteEnergyUse_kBtu', 'SteamUse_kBtu', 'YearBuilt',
       'YearsENERGYSTARCertified', 'Zip Codes', 'TotalEnergy_kBtu',
       'YearBuilt_10y', 'YearsENERGYSTARCertified_10y']
DATA_corr

In [ ]:
# Organisation des variables (FLOAT/STR,DISCRETES/CONTINUES)
lVAR_cible = ['SiteEnergyUse_kBtu', 'GHGEmissions_MetricTonsCO2e', 'SiteEnergyUse_kBtu_m2', 'GHGEmissions_MetricTonsCO2e_m2'] #Variable CIBLES 
lVAR_cible = ['SiteEnergyUse_kBtu', 'GHGEmissions_MetricTonsCO2e'] #Variable CIBLES 

lVAR_str = DATA_corr.select_dtypes(exclude=[np.floating]).columns.difference(lVAR_cible)  #Les variables str DISCRETES ET CONTINUES
lVAR_float = DATA_corr.select_dtypes(include=[np.floating]).columns.difference(lVAR_cible)  #Les variables floats DISCRETES ET CONTINUES

lVAR_discret_str = DATA_corr[lVAR_str].nunique().sort_values()[(DATA_corr.nunique()<1000)].index.difference([]) #Les variables str DISCRETES
lVAR_float_discret = DATA_corr[lVAR_float].nunique().sort_values()[(DATA_corr.nunique()<1000)].index.difference(['PropertyGFAParking', 'OtherFuelUse_kBtu', 'SPD Beats', 'SteamUse_kBtu']) #Les variables floats DISCRETES
lVAR_float_continue = lVAR_float.difference(lVAR_float_discret) #Les variables floats CONTINUES
lVAR_discret = np.concatenate((lVAR_discret_str, lVAR_float_discret), axis=None)

print('lVAR_discret ',lVAR_discret)
print('lVAR_discret_str ',lVAR_discret_str)
print('lVAR_float_discret ',lVAR_float_discret)
print('lVAR_float_continue ',lVAR_float_continue)


In [ ]:
kruskal = pd.DataFrame([], index = lVAR_discret)

for ik in kruskal.index:
    for c in lVAR_cible:
        k = pg.kruskal(dv=c, between=ik, data=DATA_corr, detailed=True)
        k = k.set_index('Source')

        #kruskal.loc[ik,c] = k.loc[[ik],['p-unc']].values
        kruskal.loc[ik,c] = k.loc[[ik], ['p-unc']].to_numpy()[0]

In [ ]:
plt.style.use('seaborn-notebook') 
plt.rcParams["figure.figsize"] = [1.5,4]
heatmap = sns.heatmap(kruskal,  annot=True, fmt='.1f', cmap='BrBG', cbar=False) #, mask=mask)
plt.savefig(direc0 + '33-kruskal.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')
plt.style.use('seaborn-poster') 


##### ANOVA hipothese 2 : https://pingouin-stats.org/generated/pingouin.homoscedasticity.html 

###### p-unc (independant)

In [ ]:
lVAR_float_ANOVA = ['CouncilDistrictCode', 'ENERGYSTARScore', 'Electricity_kBtu',
       'Latitude', 'Longitude', 'NaturalGas_kBtu', 'NumberofFloors',
       'OtherFuelUse_kBtu', 'PropertyGFABuilding_s', 'PropertyGFAParking',
        'SteamUse_kBtu', 'TotalEnergy_kBtu', 'YearBuilt',
       'YearBuilt_10y', 'YearsENERGYSTARCertified',
       'YearsENERGYSTARCertified_10y']

In [ ]:
lVAR_discret_str

In [ ]:
pUNC = pd.DataFrame([], index = lVAR_discret_str)

for c in  lVAR_cible:
    print(c)
    aov = pg.anova(dv=c, between=lVAR_discret_str.to_list(), data=DATA_corr, detailed=True)
    aov = aov.set_index('Source')
    aov = aov.loc[lVAR_discret_str.to_list(), ['p-unc']]
    aov = aov.rename(columns = {'p-unc' : c})
    pUNC = pd.concat([pUNC,aov], axis = 1)

###### np2 (contribution)

In [ ]:
np2 = pd.DataFrame([], index = lVAR_discret_str)

for c in lVAR_cible:
    print(c)
    aov = pg.anova(dv=c, between=lVAR_discret_str.to_list(), data=DATA_corr, detailed=True)
    aov = aov.set_index('Source')
    aov = aov.loc[lVAR_discret_str, ['np2']]
    aov = aov.rename(columns = {'np2' : c})
    np2 = pd.concat([np2,aov], axis = 1)

##### Correlation Chi-square (Les variables cibles sont continues -> pas faire le chi-square)

In [ ]:
lVAR_discret

expected, observed, stats = pg.chi2_independence(DATA_corr, x=lVAR_discret_str[1], y=lVAR_discret_str[2])
#observed = observed.drop('0', axis=1)
#observed.T.style.format('{0:,.0f}').background_gradient(axis=0)
observed.T.sort_values('downtown', ascending =False).style.format('{0:,.0f}').background_gradient(axis=0)

order = observed.T.sort_values('downtown', ascending =False).index.to_list()
#expected = expected.drop('0', axis=1)
expected.T.reindex(order).style.format('{0:,.0f}').background_gradient(axis=0)

stats.style.format('{0:,.1f}', subset=stats.columns[1:])

In [ ]:
chi2 = pd.DataFrame([], index = lVAR_discret, columns = lVAR_discret)
pval = pd.DataFrame([], index = lVAR_discret, columns = lVAR_discret)

for c in lVAR_discret:
    for f in lVAR_discret:
        if c != f:
            expected, observed, stats = pg.chi2_independence(DATA_corr, x=c, y=f)
            chi2.loc[f,c] = stats['chi2'].loc[0]
            pval.loc[f,c] = stats['pval'].loc[0]

chi2 = chi2.round(2).astype('float16')
pval = pval.round(2).astype('float16')

In [ ]:
plt.style.use('seaborn-notebook') 
plt.rcParams["figure.figsize"] = [5,5]
heatmap = sns.heatmap(chi2,  annot=True, fmt='.0f', cmap='BrBG', cbar=False) #, mask=mask)
plt.savefig(direc0 + '33-chi2_chi2.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

In [ ]:
heatmap = sns.heatmap(pval,  annot=True, fmt='.1f', cmap='BrBG', cbar=False) #, mask=mask)
plt.savefig(direc0 + '33-chi2_pval.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

La probabilite que la variable 'BuildingType' et 'Neighborhood'
soient independant est du 0.0% -> (la probabilité que les variables
soint dependants est de 100%) -> Correlation.


#### PCA

DATA_pca = DATA_corr.set_index('Neighborhood').loc[:,lVAR_float_continue]
DATA_pca = DATA_pca.fillna(DATA_pca.mean()) 
DATA_pca_columns = np.array(DATA_pca.columns)
DATA_pca

. # Initialize to reduce the data up to the nubmer of componentes that explains 95% of the variance.
model = pca(n_components=0.95)

.# Reduce the data towards 3 PCs
model = pca(n_components=3, normalize=True)

.# Fit transform
out = model.fit_transform(DATA_pca)

plt.style.use('seaborn-poster') 
plt.rcParams["figure.figsize"] = [6, 5]
plt.rcParams['legend.loc'] = "upper right"
#TOP = 2
#BOTTON = -6
#LEFT = -3
#RIGHT = 4
#ax.set_xlim(left=LEFT, right=RIGHT)
#ax.set_ylim(bottom=BOTTON, top=TOP)

.# Projection des individus sans SPE et  Outlier
fig, ax = model.scatter(SPE=False, hotellingt2=False, label=False, legend=True)
fig.savefig(direc0 + 'NEIG_individus_sansSPE.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

.# Projection des individus avec SPE et Outlier
fig, ax = model.scatter(SPE=True, hotellingt2=True, label=False, legend=True)
fig.savefig(direc0 + 'NEIG_individus_avecSPE.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

.# Cercle de correlation avec projection des individus
fig, ax = model.biplot(label=False, legend=True)
fig.savefig(direc0 + 'NEIG_individus_et_CercleCORRELATION.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

.# Cercle de correlation
fig, ax = model.biplot(cmap=None, label=False, legend=False)
fig.savefig(direc0 + 'NEIG_CercleCORRELATION.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

fig, ax = model.plot()
fig.savefig(direc0 + 'NEIG_Inertie.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

## DATA Exploration

In [ ]:
dfDATA2.columns

In [ ]:
# Organisation des variables (FLOAT/STR,DISCRETES/CONTINUES)
lVAR_cible = ['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)', 'SiteEnergyUse(kBtu)/m2', 'GHGEmissions(MetricTonsCO2e)/m2'] #Variable CIBLES 
lVAR_cible = ['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)'] #Variable CIBLES 

lVAR_str = dfDATA2.select_dtypes(exclude=[np.floating]).columns.difference(lVAR_cible)  #Les variables str DISCRETES ET CONTINUES
lVAR_float = dfDATA2.select_dtypes(include=[np.floating]).columns.difference(lVAR_cible)  #Les variables floats DISCRETES ET CONTINUES

lVAR_discret_str = dfDATA2[lVAR_str].nunique().sort_values()[(dfDATA2.nunique()<1000)].index.difference(['Comment']) #Les variables str DISCRETES
lVAR_continue_str  = lVAR_str.difference(lVAR_discret_str) #Les variables floats CONTINUES

lVAR_float_discret = dfDATA2[lVAR_float].nunique().sort_values()[(dfDATA2.nunique()<1000)].index.difference(['OtherFuelUse(kBtu)', 'SPD Beats', 'SteamUse(kBtu)', 'GHGEmissionsIntensity',
       'GHGEmissionsIntensity(kgCO2e/ft2)']) #Les variables floats DISCRETES
lVAR_float_continue = lVAR_float.difference(lVAR_float_discret) #Les variables floats CONTINUES

lVAR_discret = np.concatenate((lVAR_discret_str, lVAR_float_discret), axis=None)
lVAR_continue = np.concatenate((lVAR_continue_str, lVAR_float_continue), axis=None)

print('lVAR_discret_str ',lVAR_discret_str)
print('lVAR_continue_str ',lVAR_continue_str)

print('lVAR_float_discret ',lVAR_float_discret)
print('lVAR_float_continue ',lVAR_float_continue)

print('lVAR_discret ',lVAR_discret)
print('lVAR_continue ',lVAR_continue)

### Simple graphs

#### MAP

In [ ]:
# Selection until 90% percentil
Stat = dfDATA2['SiteEnergyUse(kBtu)'].describe(percentiles=[0.95])
dfDATA2_MAP = dfDATA2[dfDATA2['SiteEnergyUse(kBtu)']<Stat.iloc[5]]

In [ ]:
sns.scatterplot(data=dfDATA2_MAP, x='Longitude', y='Latitude', hue='SiteEnergyUse(kBtu)', marker='o', alpha=0.5)
plt.savefig(direc0 + '342-MAP-EnergyeUSE.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

#### 1D - Distribution

In [ ]:
plt.style.use('seaborn-notebook') 

lVARS = lVAR_float.difference(['Longitude', 'Latitude'])
lVARS = dfDATA2.loc[:,lVARS].describe(include='all').T.sort_values('max', ascending = False).index #Les variables organiser par le plus grandes aux plus petites

nc = 4
nf = 9

fig = plt.figure(figsize=(7*nc, 7*nf))
plt.subplots_adjust(wspace=0.2, hspace=0.2) #interspace entre graphiques

cc = -1
for c in lVARS:
    cc = cc + 1
    
    ax = fig.add_subplot(nf, nc, (cc+1))
    ax = sns.histplot(data=dfDATA2[c],ax = ax, common_norm=True, bins=30, kde=True)
    ax.set_title(c)
    ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '34-1D-Distribution-TOUT.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

In [ ]:
plt.style.use('seaborn-notebook') 
plt.style.use('seaborn-poster') 

lVARS = lVAR_float.difference(['Longitude', 'Latitude'])
lVARS = dfDATA2.loc[:,lVARS].describe(include='all').T.sort_values('max', ascending = False).index #Les variables organiser par le plus grandes aux plus petites
lVARS = ['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)', 'TotalEnergy(kBtu)', 'Electricity(kBtu)', 'SteamUse(kBtu)',
       'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)']
nc = 2
nf = 9

fig = plt.figure(figsize=(5*nc, 5*nf))
plt.subplots_adjust(wspace=0.3, hspace=0.3) #interspace entre graphiques

cc = -1
for c in lVARS:
    cc = cc + 1
    
    ax = fig.add_subplot(nf, nc, (cc+1))
    ax = sns.histplot(data=dfDATA2[c],ax = ax, common_norm=True, bins=30, kde=True)
    ax.set_title(c)
    ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '34-1D-Distribution-Energy.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

In [ ]:
lVAR_float.difference(['Longitude', 'Latitude'])


In [ ]:
plt.style.use('seaborn-notebook') 
plt.style.use('seaborn-poster') 

lVARS = lVAR_float.difference(['Longitude', 'Latitude'])
lVARS = dfDATA2.loc[:,lVARS].describe(include='all').T.sort_values('max', ascending = False).index #Les variables organiser par le plus grandes aux plus petites
lVARS = ['CouncilDistrictCode', 'NumberofFloors', 'ENERGYSTARScore',
       'PropertyGFABuilding(s)', 'PropertyGFAParking', 'YearBuilt_10y','YearsENERGYSTARCertified']
nc = 3
nf = 9

fig = plt.figure(figsize=(5*nc, 5*nf))
plt.subplots_adjust(wspace=0.3, hspace=0.3) #interspace entre graphiques

cc = -1
for c in lVARS:
    cc = cc + 1
    
    ax = fig.add_subplot(nf, nc, (cc+1))
    ax = sns.histplot(data=dfDATA2[c],ax = ax, common_norm=True, bins=30, kde=True)
    ax.set_title(c)
    ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '34-1D-Distribution-other.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

In [ ]:
plt.style.use('seaborn-notebook') 
plt.style.use('seaborn-poster') 

lVARS = lVAR_float.difference(['Longitude', 'Latitude'])
lVARS = dfDATA2.loc[:,lVARS].describe(include='all').T.sort_values('max', ascending = False).index #Les variables organiser par le plus grandes aux plus petites
lVARS = ['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)']
nc = 1
nf = 2

fig = plt.figure(figsize=(5*nc, 5*nf))
plt.subplots_adjust(wspace=0.3, hspace=0.3) #interspace entre graphiques

cc = -1
for c in lVARS:
    cc = cc + 1
    
    ax = fig.add_subplot(nf, nc, (cc+1))
    ax = sns.histplot(data=dfDATA2[c],ax = ax, common_norm=True, bins=30, kde=True)
    ax.set_title(c)
    ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '34-1D-Distribution-cible.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

### Combined graphs

#### 2D - Distribution

In [ ]:
plt.rcParams["figure.figsize"] = [7,7]
dfDATA_corr = dfDATA2.corr()[['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)']].sort_values('SiteEnergyUse(kBtu)', ascending=False)
# define the mask to set the values in the upper triangle to True
#mask = np.triu(np.ones_like(dfDATA_corr_ZOOM8, dtype=np.bool))
heatmap = sns.heatmap(dfDATA_corr, vmin=-1, vmax=1, annot=True, fmt='.1f', cmap='BrBG', cbar=False) #, mask=mask)
#heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16); 
plt.savefig(direc0 + '342-dfDATA2_corr.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')
plt.rcParams["figure.figsize"] = [6, 5]

##### 2D - Distribution (ZOOM)

dfDATA_corr_ZOOM8 = dfDATA2.corr()
for c in dfDATA_corr_ZOOM8.columns:
    if sum(dfDATA_corr_ZOOM8[c]>=0.65)==1:
        dfDATA_corr_ZOOM8 = dfDATA_corr_ZOOM8.drop(c, axis=0)
        dfDATA_corr_ZOOM8 = dfDATA_corr_ZOOM8.drop(c, axis=1)

dfDATA_corr_ZOOM = dfDATA2.corr()[['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)']].sort_values('SiteEnergyUse(kBtu)')
dfDATA_corr_ZOOM

plt.rcParams["figure.figsize"] = [5,5]

.# define the mask to set the values in the upper triangle to True
#mask = np.triu(np.ones_like(dfDATA_corr_ZOOM8, dtype=np.bool))
heatmap = sns.heatmap(dfDATA_corr_ZOOM8[['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)']], vmin=-1, vmax=1, annot=True, fmt='.1f', cmap='BrBG', cbar=False) #, mask=mask)
#heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16); 
plt.savefig(direc0 + '342-dfDATA2_corr_ZOOM.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')
plt.rcParams["figure.figsize"] = [6, 5]


### Relations des Variables CONTINUE avec les variables CIBLES

In [ ]:
lVAR_sns = np.concatenate((lVAR_float_continue.difference(['Latitude','Longitude','GHGEmissions(MetricTonsCO2e)/m2', 'SiteEnergyUse(kBtu)/m2']), lVAR_cible), axis=None)


In [ ]:
lVAR_sns

In [ ]:
plt.style.use('seaborn-notebook') 
lVAR_sns = np.concatenate((lVAR_float_continue.difference(['Latitude','Longitude''GHGEmissions(MetricTonsCO2e)/m2', 'SiteEnergyUse(kBtu)/m2']), lVAR_cible), axis=None)
lVAR_sns = ['PropertyGFABuilding(s)', 'PropertyGFAParking', 'SPD Beats', 
            'SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)']
ax = sns.pairplot(np.log10(dfDATA2[lVAR_sns]), x_vars=lVAR_cible, diag_kind= 'kde',  plot_kws=dict(marker=".", linewidth=0))
#ax.set_xscale('log')
#ax.set_yscale('log')
ax.savefig(direc0 + '34-DISTRIBUTION_Cont_0.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

In [ ]:
lVAR_sns = np.concatenate((lVAR_float_continue.difference(['Latitude','Longitude''GHGEmissions(MetricTonsCO2e)/m2', 'SiteEnergyUse(kBtu)/m2']), lVAR_cible), axis=None)
lVAR_sns = ['Electricity(kBtu)', 'NaturalGas(kBtu)', 'OtherFuelUse(kBtu)', 'SteamUse(kBtu)', 'TotalEnergy(kBtu)',
            'SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)']
ax = sns.pairplot(np.log10(dfDATA2[lVAR_sns]), x_vars=lVAR_cible, diag_kind= 'kde',  plot_kws=dict(marker=".", linewidth=0))
#ax.set_xscale('log')
#ax.set_yscale('log')
ax.savefig(direc0 + '34-DISTRIBUTION_Cont_1.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

### Relations des Variables DISCRETES avec les variables CIBLES

#### Organisation des variables (FLOAT/STR,DISCRETES/CONTINUES)

In [ ]:
# Organisation des variables (FLOAT/STR,DISCRETES/CONTINUES)
lVAR_cible = ['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)', 'SiteEnergyUse(kBtu)/m2', 'GHGEmissions(MetricTonsCO2e)/m2'] #Variable CIBLES 
lVAR_cible = ['SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)'] #Variable CIBLES 

lVAR_str = dfDATA2.select_dtypes(exclude=[np.floating]).columns.difference(lVAR_cible)  #Les variables str DISCRETES ET CONTINUES
lVAR_float = dfDATA2.select_dtypes(include=[np.floating]).columns.difference(lVAR_cible)  #Les variables floats DISCRETES ET CONTINUES

lVAR_discret_str = dfDATA2[lVAR_str].nunique().sort_values()[(dfDATA2.nunique()<500)].index.difference([]) #Les variables str DISCRETES
lVAR_float_discret = dfDATA2[lVAR_float].nunique().sort_values()[(dfDATA2.nunique()<500)].index.difference(['OtherFuelUse(kBtu)','PropertyGFAParking', 'SPD Beats', 'SteamUse(kBtu)']) #Les variables floats DISCRETES
lVAR_float_continue = lVAR_float.difference(lVAR_float_discret) #Les variables floats CONTINUES
lVAR_discret = np.concatenate((lVAR_discret_str, lVAR_float_discret), axis=None)
print('lVAR_discret ',lVAR_discret)
print('lVAR_discret_str ',lVAR_discret_str)
print('lVAR_float_discret ',lVAR_float_discret)
print('lVAR_float_continue ',lVAR_float_continue)


#### Reorganisation DF pour SNS

In [ ]:
lVAR_SNS = np.concatenate((lVAR_cible, lVAR_discret_str, lVAR_float_discret), axis=None)
lVAR_SNS

In [ ]:
dDATA_cible = pd.melt(dfDATA2, id_vars=lVAR_SNS)
dDATA_cible

In [ ]:
lVAR_discret[np.isin(lVAR_discret, ['NumberofFloors','Zip Codes'], invert=True)]

In [ ]:
lVAR_discret

#### Cible - Discret

In [ ]:
plt.style.use('seaborn-notebook') 

nc = 2
nf = 20

fig = plt.figure(figsize=(10*nc, 7*nf))
plt.subplots_adjust(wspace=0.1, hspace=0.2*2) #interspace entre graphiques

cc = -1
for icd in lVAR_discret[np.isin(lVAR_discret, ['NumberofFloors','NumberofBuildings', 'YearsENERGYSTARCertified_10y', 'ENERGYSTARScore', 'NumberofFloors', 'YearBuilt'], invert=True)]:
    for ici in lVAR_cible[:2]:
        cc = cc + 1

        ax = fig.add_subplot(nf, nc, (cc+1))
        g = sns.boxplot(data=dDATA_cible, y=ici, x=icd, showfliers=False, showmeans = True, ax = ax) #)
        g.set_xticklabels(g.get_xticklabels(), rotation=-90)

        ax.set_title(icd)
        #ax.set_ylabel('')
        ax.set_xlabel('')
        #ax.set_yscale('log')
        #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '35-CIBLE-Discret.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

In [ ]:
lVAR_discret

In [ ]:
plt.style.use('seaborn-notebook') 
plt.style.use('seaborn-poster') 

nc = 2
nf = 20

fig = plt.figure(figsize=(10*nc, 7*nf))
plt.subplots_adjust(wspace=0.2, hspace=0.5*2) #interspace entre graphiques

cc = -1
for icd in ['BuildingType','PrimaryPropertyType']:
    for ici in lVAR_cible[:2]:
        cc = cc + 1

        ax = fig.add_subplot(nf, nc, (cc+1))
        g = sns.boxplot(data=dDATA_cible, y=ici, x=icd, showfliers=False, showmeans = True, ax = ax) #)
        g.set_xticklabels(g.get_xticklabels(), rotation=-90)

        ax.set_title(icd)
        #ax.set_ylabel('')
        ax.set_xlabel('')
        #ax.set_yscale('log')
        #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '35-CIBLE-Discret-Build-Property.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

In [ ]:
plt.style.use('seaborn-notebook') 
plt.style.use('seaborn-poster') 

nc = 2
nf = 20

fig = plt.figure(figsize=(10*nc, 7*nf))
plt.subplots_adjust(wspace=0.2, hspace=0.2*2) #interspace entre graphiques

cc = -1
for icd in ['Neighborhood','CouncilDistrictCode', 'Zip Codes']:
    for ici in lVAR_cible[:2]:
        cc = cc + 1

        ax = fig.add_subplot(nf, nc, (cc+1))
        g = sns.boxplot(data=dDATA_cible, y=ici, x=icd, showfliers=False, showmeans = True, ax = ax) #)
        g.set_xticklabels(g.get_xticklabels(), rotation=-90)

        ax.set_title(icd)
        #ax.set_ylabel('')
        ax.set_xlabel('')
        #ax.set_yscale('log')
        #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '35-CIBLE-Discret-Neig-Z-code.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

In [ ]:
plt.style.use('seaborn-notebook') 
plt.style.use('seaborn-poster') 

nc = 2
nf = 20

fig = plt.figure(figsize=(10*nc, 7*nf))
plt.subplots_adjust(wspace=0.2, hspace=0.2*2) #interspace entre graphiques

cc = -1
for icd in ['YearBuilt_10y','ENERGYSTARScore', 'NumberofFloors']:
    for ici in lVAR_cible[:2]:
        cc = cc + 1

        ax = fig.add_subplot(nf, nc, (cc+1))
        g = sns.boxplot(data=dDATA_cible, y=ici, x=icd, showfliers=False, showmeans = True, ax = ax) #)
        g.set_xticklabels(g.get_xticklabels(), rotation=-90)

        ax.set_title(icd)
        #ax.set_ylabel('')
        ax.set_xlabel('')
        #ax.set_yscale('log')
        #ax.set_xscale('symlog', linthresh=10000000)
plt.show()
fig.savefig(direc0 + '35-CIBLE-Discret-year-e-floor.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

m = Basemap(width=120000,height=90000,projection='aeqd',
            resolution=None,lat_0=dfDATA2['Latitude'].mean(),lon_0=dfDATA2['Longitude'].mean())


m.bluemarble()
x, y = m(dfDATA2['Longitude'],dfDATA2['Latitude'])

m.scatter(x,y,10,marker='o',color='k')


plt.show()

ax.set(ylim=(dfDATA2['Latitude'].min(),dfDATA2['Latitude'].max()))
ax.set(xlim=(dfDATA2['Longitude'].min(),dfDATA2['Longitude'].max()))

In [ ]:
lVAR_discret

In [ ]:
plt.style.use('seaborn-notebook') 
plt.rcParams["figure.figsize"] = [6, 5]

nc = 2
nf = 20

fig = plt.figure(figsize=(6*nc*2, 5*nf*2))
plt.subplots_adjust(wspace=0.2, hspace=0.2) #interspace entre graphiques

cc = -1
for icd in ['Neighborhood', 'Zip Codes', 'CouncilDistrictCode',]:
    cc = cc + 1

    ax = fig.add_subplot(nf, nc, (cc+1))
    g = sns.scatterplot(data=dfDATA2, x='Longitude', y='Latitude', hue=icd, marker='.', alpha=0.5)
    #g.set_xticklabels(g.get_xticklabels(), rotation=-90)

    ax.set_title(icd)
    #ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_xscale('symlog', linthresh=10000000)
    
    
plt.show()
fig.savefig(direc0 + '35-MAP-zip.pdf', bbox_inches ="tight", pad_inches = 0.1, format='pdf')

### Link and brush

### Nongraphical techniques

In [ ]:
lVAR_cible[:2]

# Export to CSV

In [ ]:
dfDATA2.columns

In [ ]:
lVAR_ML = np.concatenate((lVAR_cible[:2], ['PropertyGFABuilding(s)', 'NumberofFloors','PrimaryPropertyType', 'Neighborhood', 'YearBuilt_10y', 'ENERGYSTARScore']), axis=None)
dfML = dfDATA2[lVAR_ML]

In [ ]:
dfML.to_csv(direc0 + 'dfML.csv', index=True)
dfML.head()

In [ ]:
dfML.dtypes[0]
